In [13]:
import re
from urllib.request import Request, urlopen
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
url = 'https://ati.utexas.edu/companies/'

In [15]:
# handle 403 Forbidden
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page_content = urlopen(req)
soup = BeautifulSoup(page_content, 'html.parser')

Instance of post:
```html
<a class="elementor-post__thumbnail__link" href="https://ati.utexas.edu/company/amt/">
```
Go to that link.

In [21]:
def parse_post(link):
  link_content = BeautifulSoup(urlopen(link), 'html.parser')
  containers = link_content.find_all('div', class_='elementor-image')
  descs = link_content.find_all('div', class_='one_third')
  titles = link_content.find_all('h1', class_='page-header-title')
  name = containers[0].img['alt'].title()
  try:
    link = containers[1].a['href']
  except:
    link = ''
    print(name + ' has no link')
  title = titles[0].text
  try:
    desc = descs[0].p.text
  except:
    desc = ''
    print(name + ' has no description')
  ret = {'Name': name, 'Desc': desc, 'Link': link, 'Title': title}
  return ret

In [22]:
link = posts[10]['href']  # individual link to go into
parse_post(link)

New Dominion Enterprises has no description


{'Name': 'New Dominion Enterprises',
 'Desc': '',
 'Link': 'https://www.newdominionenterprises.com/',
 'Title': 'New Dominion Enterprises – Safer and More Durable Lithium Batteries'}

In [16]:
posts = soup.find_all('a', class_='elementor-post__thumbnail__link')

In [28]:
df = pd.DataFrame(columns=['Name', 'Desc', 'Link', 'Title'])
for post in posts:
  link = post['href']
  df = df.append(parse_post(link), ignore_index=True)

Analog Medical has no link
Atom Mines has no description
Big Wheelbarrow has no link
Earthly Labs has no description
Epg Eco Team has no link
New Dominion Enterprises has no description


The resultant dataframe

In [29]:
df.head()

,Name,Desc,Link,Title
0,Analog Medical,Analog medical technologies’ (AMT) mission is ...,,Analog Medical – Empowering Patients with Chro...
1,Atom Mines,,https://www.atommines.com/,Atom Mines – Separated Isotopes and Tailored I...
2,Big Wheelbarrow,We connect wholesale food buyers with local fa...,,Big Wheelbarrow – Connects Wholesale Food Buye...
3,Earthly Labs,,https://www.earthlylabs.com/,Earthly – Accessible Carbon Capture Technology
4,Epg Eco Team,"EGP Eco’s environmentally-friendly, patented t...",,EGP Eco – H2O Reduction Technology


In [30]:
df.to_csv('ati.csv', index=False)